## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Hobart,AU,-42.8794,147.3294,58.23,79,75,5.75,moderate rain
1,1,Castro,BR,-24.7911,-50.0119,59.76,90,0,4.72,clear sky
2,2,Pisco,PE,-13.7000,-76.2167,61.02,82,100,12.66,overcast clouds
3,3,Barrow,US,71.2906,-156.7887,41.02,93,100,13.80,overcast clouds
4,4,Kenda,IN,23.2000,86.5333,79.07,84,74,5.10,broken clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) &
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
4,4,Kenda,IN,23.2000,86.5333,79.07,84,74,5.10,broken clouds
5,5,Jamestown,US,42.0970,-79.2353,78.44,47,0,10.36,clear sky
8,8,Rikitea,PF,-23.1203,-134.9692,75.33,77,50,16.87,light rain
9,9,Portland,US,45.5234,-122.6762,80.62,56,20,11.50,few clouds
10,10,Hilo,US,19.7297,-155.0900,85.28,84,100,5.75,overcast clouds
...,...,...,...,...,...,...,...,...,...,...
694,694,Najran,SA,17.4924,44.1277,83.05,41,72,3.94,broken clouds
697,697,Griffith,US,41.5284,-87.4237,84.11,49,4,5.93,clear sky
699,699,Aitape,PG,-3.1374,142.3475,85.66,64,4,3.91,clear sky
702,702,Boa Vista,BR,2.8197,-60.6733,80.58,78,40,2.30,scattered clouds


In [13]:
# 4a. Determine if there are any empty rows.
if preferred_cities_df.isnull().values.any() == True:
    print("Null values present in the Preferred cities list")

In [14]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
if preferred_cities_df.isnull().values.any() == True:
    preferred_cities_df = preferred_cities_df.dropna()
clean_df = preferred_cities_df

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
4,Kenda,IN,79.07,broken clouds,23.2000,86.5333,
5,Jamestown,US,78.44,clear sky,42.0970,-79.2353,
8,Rikitea,PF,75.33,light rain,-23.1203,-134.9692,
9,Portland,US,80.62,few clouds,45.5234,-122.6762,
10,Hilo,US,85.28,overcast clouds,19.7297,-155.0900,
13,Vaini,TO,77.16,scattered clouds,-21.2000,-175.2000,
15,Spencer,US,87.91,few clouds,43.1414,-95.1444,
24,Ibra,OM,79.18,clear sky,22.6906,58.5334,
25,Ishigaki,JP,86.13,few clouds,24.3448,124.1572,
31,Kapaa,US,84.18,light rain,22.0752,-159.3190,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()  
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
hotel_df.head(10)
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
4,Kenda,IN,79.07,broken clouds,23.2000,86.5333,Dilip kumar mudi
5,Jamestown,US,78.44,clear sky,42.0970,-79.2353,Hampton Inn & Suites Jamestown
8,Rikitea,PF,75.33,light rain,-23.1203,-134.9692,People ThankYou
9,Portland,US,80.62,few clouds,45.5234,-122.6762,Embassy Suites by Hilton Portland Downtown
10,Hilo,US,85.28,overcast clouds,19.7297,-155.0900,Hilo Hawaiian Hotel
13,Vaini,TO,77.16,scattered clouds,-21.2000,-175.2000,Keleti Beach Resort
15,Spencer,US,87.91,few clouds,43.1414,-95.1444,Reston Inn Suites
24,Ibra,OM,79.18,clear sky,22.6906,58.5334,مزرعة راحة النفوس المنزفة
25,Ishigaki,JP,86.13,few clouds,24.3448,124.1572,Hotel Peace Land Ishigakijima
31,Kapaa,US,84.18,light rain,22.0752,-159.3190,Sheraton Kauai Coconut Beach Resort


In [11]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()
clean_hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
4,Kenda,IN,79.07,broken clouds,23.2000,86.5333,Dilip kumar mudi
5,Jamestown,US,78.44,clear sky,42.0970,-79.2353,Hampton Inn & Suites Jamestown
8,Rikitea,PF,75.33,light rain,-23.1203,-134.9692,People ThankYou
9,Portland,US,80.62,few clouds,45.5234,-122.6762,Embassy Suites by Hilton Portland Downtown
10,Hilo,US,85.28,overcast clouds,19.7297,-155.0900,Hilo Hawaiian Hotel
...,...,...,...,...,...,...,...
694,Najran,SA,83.05,broken clouds,17.4924,44.1277,محمد هادي ال فطيح
697,Griffith,US,84.11,clear sky,41.5284,-87.4237,Apartment above Depot Bar
699,Aitape,PG,85.66,clear sky,-3.1374,142.3475,
702,Boa Vista,BR,80.58,scattered clouds,2.8197,-60.6733,Aipana Plaza Hotel


In [12]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [15]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [16]:
# 11a. Add a marker layer for each city to the map. 
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))